#### Prepared for Gabor's Data Analysis

### Data Analysis for Business, Economics, and Policy
by Gabor Bekes and  Gabor Kezdi
 
Cambridge University Press 2021

**[gabors-data-analysis.com ](https://gabors-data-analysis.com/)**

 License: Free to share, modify and use for educational purposes. 
 Not to be used for commercial purposes.


### CHAPTER 20

**CH20B Fine tuning social media advertising**

using the ab-test-social-media dataset

version 0.9 2020-09-11

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import os
import sys

import numpy as np
import pandas as pd
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
from statsmodels.stats.power import TTestIndPower
from scipy.stats import ttest_ind

In [2]:
# Current script folder
current_path = os.getcwd()
dirname = "/".join(current_path.split("/")[:-2]) + "/"

# location folders
data_in = dirname + "da_data_repo/ab-test-social-media/raw"
data_out = dirname + "da_case_studies/ch20-ab-test-social-media/"
output = dirname + "da_case_studies/ch20-ab-test-social-media/output/"

func = dirname + "da_case_studies/ch00-tech-prep/"
sys.path.append(func)

## Part I

sample size calculation

sample size calculation with planned rates

In [3]:
clickthrough = 0.01

conversion = 0.05

proportionA = clickthrough * conversion

proportionB = proportionA * 1.2

es = sms.proportion_effectsize(proportionA, proportionB)
# es = proportionB - proportionA
TTestIndPower().solve_power(es, power=0.8, alpha=0.05) * 2

1722229.7535679373

In [4]:
clickthrough = 0.0032

conversion = 0.0082

proportionA = clickthrough * conversion

proportionB = proportionA * 1.2

es = sms.proportion_effectsize(proportionA, proportionB)
# es = proportionB - proportionA
TTestIndPower().solve_power(es, power=0.8, alpha=0.05) * 2

32833931.083989523

## Part II

p-value of tests

In [5]:
summary_data = pd.read_excel(data_in+"/ab-test-summary.xlsx").set_index("action_type")

In [6]:
summary_data

,show,clicks,action
action_type,,,
Action A,1000000,3323,32
Action B,1000000,3128,21


In [7]:
data = pd.DataFrame(0, columns=["type_b", "clicks", "action"], index=range(0, 2000000))

data.loc[0:999999, "type_b"] = 1

data.loc[0 : summary_data.loc["Action A", "clicks"] - 1, "clicks"] = 1
data.loc[1000000 : 1000000 + summary_data.loc["Action B", "clicks"] - 1, "clicks"] = 1
data.loc[0 : summary_data.loc["Action A", "action"] - 1, "action"] = 1
data.loc[1000000 : 1000000 + summary_data.loc["Action B", "action"] - 1, "action"] = 1

In [8]:
data.groupby(["type_b","clicks"]).count()

action
type_b clicks        
0      0       996872
       1         3128
1      0       996677
       1         3323

In [9]:
data.groupby(["type_b","action"]).count()

clicks
type_b action        
0      0       999979
       1           21
1      0       999968
       1           32

In [10]:
smf.ols("clicks ~ 1 + type_b", data).fit(cov_type="HC0").summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 clicks   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     5.914
Date:                Mon, 04 Jan 2021   Prob (F-statistic):             0.0150
Time:                        19:32:03   Log-Likelihood:             2.9020e+06
No. Observations:             2000000   AIC:                        -5.804e+06
Df Residuals:                 1999998   BIC:                        -5.804e+06
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0031   5.58e-05     56.016      0.000       0.003       0.003
type_b         0.0002   8.02e-05      2.432      0.015    3.78e-05       0.000
==============================================================================
Omnibus:                  4075093.883   Durbin-Watson:                   0.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       7855996717.432
Skew:                          17.522   Prob(JB):                         0.00
Kurtosis:                     308.031   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [11]:
smf.ols("action ~ 1 + type_b", data).fit(cov_type="HC0").summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 action   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.283
Date:                Mon, 04 Jan 2021   Prob (F-statistic):              0.131
Time:                        19:32:04   Log-Likelihood:             7.7005e+06
No. Observations:             2000000   AIC:                        -1.540e+07
Df Residuals:                 1999998   BIC:                        -1.540e+07
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     2.1e-05   4.58e-06      4.583      0.000     1.2e-05       3e-05
type_b        1.1e-05   7.28e-06      1.511      0.131   -3.27e-06    2.53e-05
===============================================================================
Omnibus:                  9693574.572   Durbin-Watson:                    0.057
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   118646783578266.656
Skew:                         194.249   Prob(JB):                          0.00
Kurtosis:                   37733.763   Cond. No.                          2.62
===============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""